In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
import pandas as pd

SALARY_NAME = "hh_vacancies_salary_model.parquet"

DRIVE_DIR = Path("/content/drive/MyDrive/ColabNotebooks/FinalProject/ProcessedDatasets")

salary_path = DRIVE_DIR / SALARY_NAME

if salary_path.exists():
    df_salary = pd.read_parquet(salary_path)

else:
    from google.colab import files
    uploaded = files.upload()
    df_salary = pd.read_parquet(SALARY_NAME)

# **Гипотеза 1. Влияние типа графика работы на заработную плату**

В рамках исследования была проверена гипотеза о различии уровня заработной платы в зависимости от типа графика работы, указанного в вакансии.

*В анализ были включены следующие категории графика (schedule), представленные в классификации hh.ru:*

* fullDay

* remote

* shift

* flexible

* flyInFlyOut

*В качестве целевой переменной использовалась медианная оценка заработной платы (salary_mid).*

**Формулировка гипотез**

1.   H0: распределения заработной платы одинаковы для всех типов графика работы

2.   H1: хотя бы одна группа отличается

Поскольку сравнивается более двух независимых групп, а распределение заработной платы является асимметричным и содержит выбросы, для проверки гипотезы был применён непараметрический критерий Краскела–Уоллиса.

В случае выявления статистически значимых различий предполагается дополнительный анализ для определения групп, между которыми наблюдаются наибольшие расхождения.

In [ ]:
import scipy.stats as stats

groups = [
    df_salary[df_salary["schedule"] == sched]["salary_mid"]
    for sched in df_salary["schedule"].unique()
]

h_stat, p_value = stats.kruskal(*groups)

print("H statistic:", h_stat)
print("p-value:", p_value)

medians = df_salary.groupby("schedule")["salary_mid"].median().sort_values()
print("\nMedians by schedule:")
print(medians)

counts = df_salary["schedule"].value_counts()
print("\nCounts by schedule:")
print(counts)

H statistic: 810.6652097005848
p-value: 3.7598286278433077e-174

Medians by schedule:
schedule
shift           87000.0
flexible       100000.0
remote         105000.0
fullDay        117000.0
flyInFlyOut    189250.0
Name: salary_mid, dtype: float64

Counts by schedule:
schedule
fullDay        9103
remote         1881
shift          1454
flexible       1359
flyInFlyOut     334
Name: count, dtype: int64


# Вывод

В результате применения критерия Краскела–Уоллиса были выявлены статистически значимые различия в уровне заработной платы в зависимости от типа графика работы (H = 810.67, p < 0.001).

Наиболее высокий уровень оплаты наблюдается при вахтовом графике, тогда как сменный график характеризуется минимальной медианной заработной платой.

Таким образом, тип графика работы является значимым фактором, влияющим на уровень оплаты труда.

# **Гипотеза 2. Влияние уровня опыта на заработную плату**

В рамках исследования была проверена гипотеза о различии уровня заработной платы в зависимости от требуемого опыта работы.

*В анализ были включены следующие категории опыта:*

* noExperience

* between1And3

* between3And6

* moreThan6

*В качестве целевой переменной использовалась медианная оценка заработной платы (salary_mid).*

**Формулировка гипотез**

1.   H0: распределения заработных плат во всех группах опыта одинаковы

2.   H1: хотя бы одна группа отличается

Поскольку сравнивается более двух независимых групп, а распределение заработной платы является асимметричным, был применён непараметрический критерий Краскела–Уоллиса.

In [ ]:
import numpy as np
import scipy.stats as stats

groups = []
for exp in df_salary["experience"].unique():
    groups.append(
        df_salary[df_salary["experience"] == exp]["salary_mid"]
    )

h_stat, p_value = stats.kruskal(*groups)

print("H statistic:", h_stat)
print("p-value:", p_value)

medians = df_salary.groupby("experience")["salary_mid"].median()
print("\nMedians by experience:")
print(medians)

H statistic: 2956.2772778352764
p-value: 0.0

Medians by experience:
experience
between1And3    104400.0
between3And6    150000.0
moreThan6       200000.0
noExperience     84825.0
Name: salary_mid, dtype: float64


# Вывод

Критерий Краскела–Уоллиса показал статистически значимые различия в уровне заработной платы в зависимости от опыта (H = 2956.28, p < 0.001).

Медианная зарплата по группам:

noExperience — 84 825 руб.

between1And3 — 104 400 руб.

between3And6 — 150 000 руб.

moreThan6 — 200 000 руб.

Наблюдается последовательный рост заработной платы с увеличением опыта. Полученные различия являются не только статистически значимыми, но и практически существенными.

# **Гипотеза 3. Влияние навыка PostgreSQL на уровень заработной платы**

В рамках исследования была проверена гипотеза о различии уровня заработной платы в зависимости от наличия навыка PostgreSQL в требованиях вакансии.

*Были сформированы две независимые группы:*

* вакансии, содержащие навык PostgreSQL

* вакансии без указания данного навыка

*В качестве целевой переменной использовалась медианная оценка заработной платы (salary_mid).*

**Формулировка гипотез**

1.   H0: распределения заработных плат в группах одинаковы

2.   H1: распределения различаются

Поскольку сравниваются две независимые группы, а распределение заработной платы является асимметричным, был применён непараметрический критерий Манна–Уитни.
Дополнительно рассчитан размер эффекта (Cliff’s delta).

In [ ]:
import numpy as np
import scipy.stats as stats

df_salary["has_postgresql"] = df_salary["key_skills"].fillna("").str.lower().str.contains("postgresql")

with_pg = df_salary[df_salary["has_postgresql"] == True]["salary_mid"]
without_pg = df_salary[df_salary["has_postgresql"] == False]["salary_mid"]

u_stat, p_value = stats.mannwhitneyu(with_pg, without_pg, alternative="two-sided")

print("U statistic:", u_stat)
print("p-value:", p_value)

print("Median with PostgreSQL:", np.median(with_pg))
print("Median without PostgreSQL:", np.median(without_pg))

def cliffs_delta(x, y):
    nx = len(x)
    ny = len(y)
    gt = 0
    lt = 0
    for xi in x:
        gt += np.sum(xi > y)
        lt += np.sum(xi < y)
    return (gt - lt) / (nx * ny)

delta = cliffs_delta(with_pg.values, without_pg.values)
print("Cliff's delta:", delta)

U statistic: 738550.0
p-value: 4.5046211722577184e-13
Median with PostgreSQL: 180000.0
Median without PostgreSQL: 110000.0
Cliff's delta: 0.5007061070641186


# Вывод

Критерий Манна–Уитни показал статистически значимые различия в уровне заработной платы в зависимости от наличия навыка PostgreSQL (U = 738 550.0, p < 0.001).

Медианная зарплата:

с PostgreSQL — 180 000 руб.

без PostgreSQL — 110 000 руб.

Размер эффекта (Cliff’s δ = 0.501) соответствует большому эффекту.

Таким образом, наличие навыка PostgreSQL связано с существенно более высоким уровнем заработной платы.

#**Гипотеза 4. Влияние профессиональной роли на уровень заработной платы**

В рамках исследования была проверена гипотеза о наличии различий в уровне заработной платы в зависимости от профессиональной роли, указанной в вакансии.

*В качестве переменных использовались:*

* professional_role_main — основная профессиональная роль вакансии

* salary_mid — медианная оценка заработной платы

**Формулировка гипотез**

1.   H0: распределения заработной платы одинаковы во всех профессиональных ролях

2.   H1: хотя бы одна профессиональная роль характеризуется отличающимся распределением заработной платы

Поскольку сравнивается более двух независимых групп, а распределение заработной платы является асимметричным и содержит выбросы, для проверки гипотезы был применён непараметрический критерий Краскела–Уоллиса.

In [ ]:
import scipy.stats as stats
import numpy as np

print("Количество уникальных ролей:",
      df_salary["professional_role_main"].nunique())

role_counts = df_salary["professional_role_main"].value_counts()
print("\nТоп-10 ролей по количеству вакансий:")
print(role_counts.head(10))

valid_roles = role_counts[role_counts >= 100].index
df_roles = df_salary[df_salary["professional_role_main"].isin(valid_roles)]

groups = [
    df_roles[df_roles["professional_role_main"] == role]["salary_mid"]
    for role in df_roles["professional_role_main"].unique()
]

h_stat, p_value = stats.kruskal(*groups)

print("\nH statistic:", h_stat)
print("p-value:", p_value)

medians = (
    df_roles.groupby("professional_role_main")["salary_mid"]
    .median()
    .sort_values(ascending=False)
)

print("\nТоп-10 ролей по медианной зарплате:")
print(medians.head(10))

Количество уникальных ролей: 173

Топ-10 ролей по количеству вакансий:
professional_role_main
Другое                                                  1363
Менеджер по продажам, менеджер по работе с клиентами    1315
Администратор                                            663
Бухгалтер                                                485
Продавец-консультант, продавец-кассир                    395
Менеджер по маркетингу, интернет-маркетолог              351
Водитель                                                 347
Упаковщик, комплектовщик                                 335
Врач                                                     288
Секретарь, помощник руководителя, ассистент              278
Name: count, dtype: int64

H statistic: 2331.5416342738786
p-value: 0.0

Топ-10 ролей по медианной зарплате:
professional_role_main
Программист, разработчик                                190700.00
Руководитель проектов                                   174000.00
Курьер                          

# Вывод
Критерий Краскела–Уоллиса показал статистически значимые различия в уровне заработной платы в зависимости от профессиональной роли (H = 2331.54, p < 0.001). Полученные результаты свидетельствуют о существенных различиях в уровне оплаты труда между профессиональными ролями.

#**Итоговый вывод**

В результате проведённого статистического анализа установлено, что уровень заработной платы в исследуемых вакансиях значимо зависит от ряда факторов.

Наиболее выраженное влияние оказывает требуемый опыт работы: наблюдается последовательный и существенный рост медианной заработной платы по мере увеличения уровня опыта.

Значимые различия также выявлены между профессиональными ролями, что отражает структурную сегментацию рынка труда и различия в функциональной специализации.

Тип графика работы оказывает статистически значимое влияние на уровень оплаты, при этом наиболее высокие значения наблюдаются при вахтовом формате работы.

Отдельно было подтверждено, что наличие специализированного технического навыка (PostgreSQL) связано с существенным увеличением уровня заработной платы и характеризуется большим размером эффекта.

Таким образом, ключевыми детерминантами уровня заработной платы в рассматриваемой выборке являются профессиональный опыт, функциональная роль и наличие специализированных технических компетенций.